# Imports

In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import zipfile as zf
from glob import glob
# from keras.saving.hdf5_format import save_attributes_to_hdf5_group
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration, AutoTokenizer, TFBertModel, AutoModel, BertTokenizer

2023-06-14 01:10:23.034061: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 01:10:23.673976: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 01:10:23.675264: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 01:10:29.657853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# !pip install semanticscholar
# !pip install metapub
# !pip install sentencepiece
import sentencepiece
from metapub import PubMedFetcher
from semanticscholar import SemanticScholar
from metapub import FindIt
import requests
import urllib
import json

2023-06-14 01:10:54 w210-therapeutic-accelerator metapub.config[18022] WARNING NCBI_API_KEY was not set.


## Tokenizer & Model Imports

In [3]:
bio_bert_model = AutoModel.from_pretrained("gsarti/biobert-nli")

In [4]:
bio_bert_tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")

In [5]:
original_bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
T5Abstract_model = TFT5ForConditionalGeneration.from_pretrained('t5-base')

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
T5tokens = T5Tokenizer.from_pretrained('t5-base')

/home/brathwaiten16_berkeley_edu/anaconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


## Data Import

In [7]:
# !gunzip ../zip_file1.gz

In [8]:
# Get info about the latest release
latest_release = requests.get("http://api.semanticscholar.org/datasets/v1/release/2023-05-30", 
                              headers={'x-api-key':'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'}).json()

# Get info about past releases
dataset_ids = requests.get("http://api.semanticscholar.org/datasets/v1/release").json()
earliest_release = requests.get(f"http://api.semanticscholar.org/datasets/v1/release/{dataset_ids[0]}").json()

# Get info about the papers dataset
papers = requests.get("http://api.semanticscholar.org/datasets/v1/release/latest/dataset/papers",
                      headers={'x-api-key':'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'}).json()

In [9]:
print(json.dumps(latest_release['datasets'][7], indent=1))

{
 "name": "s2orc",
 "description": "Full-body paper text parsed from open-access PDFs. Identifies structural elements such as paragraphs, sections, and bibliography entries.\n5M records in 30 4GB files.",
 "README": "Semantic Scholar Academic Graph Datasets\n\nThe \"s2orc\" dataset contains parsed full-body text from selected papers.\n\nA subset of this data was previously released (in a different format) as S2ORC https://github.com/allenai/s2orc\n\nThe body text is parsed from PDF documents using Grobid, documented at https://grobid.readthedocs.io.\nIts output is converted from XML into a single string with a set of annotation spans.\n\nSCHEMA\n - externalIds: IDs of this paper in different catalogs\n - content:\n   - source:\n\t   - pdfUrls: URLs to the PDF\n\t   - oaInfo: license/url/status information from Unpaywall\n   - text: Full body text as a single string\n   - annotations: Annotated spans of the full body text\n\n\nLICENSE\nThis collection is licensed under ODC-BY. (https:/

In [10]:
earliest_release

{'release_id': '2022-05-10',
 'README': 'Semantic Scholar Academic Graph Datasets\n\nThese datasets provide a variety of information about research papers taken from a snapshot in time of the Semantic Scholar corpus.\n\nThis site is provided by The Allen Institute for Artificial Intelligence (“AI2”) as a service to the\nresearch community. The site is covered by AI2 Terms of Use and Privacy Policy. AI2 does not claim\nownership of any materials on this site unless specifically identified. AI2 does not exercise editorial\ncontrol over the contents of this site. AI2 respects the intellectual property rights of others. If\nyou believe your copyright or trademark is being infringed by something on this site, please follow\nthe "DMCA Notice" process set out in the Terms of Use (https://allenai.org/terms).\n\nSAMPLE DATA ACCESS\nSample data files can be downloaded with the following UNIX command:\n\nfor f in $(curl https://s3-us-west-2.amazonaws.com/ai2-s2ag/samples/MANIFEST.txt)\n  do curl 

In [11]:
papers

{'name': 'papers',
 'description': 'The core attributes of a paper (title, authors, date, etc.).\n200M records in 30 1.5GB files.',
 'README': 'Semantic Scholar Academic Graph Datasets\n\nThe "papers" dataset provides core metadata about papers.\n\nSCHEMA\nSee https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data\n\nThis dataset does not contain information about a paper\'s references or citations.\nInstead, join with citingPaperId/citedPaperId from the "citations" dataset.\n\nLICENSE\nThis collection is licensed under ODC-BY. (https://opendatacommons.org/licenses/by/1.0/)\n\nBy downloading this data you acknowledge that you have read and agreed to all the terms in this license.\n\nATTRIBUTION\nWhen using this data in a product or service, or including data in a redistribution, please cite the following paper:\n\nBibTex format:\n@misc{https://doi.org/10.48550/arxiv.2301.10140,\n  title = {The Semantic Scholar Open Data Platform},\n  author = {Kinney, Rodney and Anastasiades, Ch

In [12]:
sch = SemanticScholar()

sch = SemanticScholar(api_key='jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB')

paper = sch.get_paper('d0bc1501ae6f54dd16534e651d90d2aeeeb1cfc1')  #'10.1093/mind/lix.236.433')

paper.referenceCount #number of references in this one, can help us create some sort of pagerank type algo
paper.citationCount #number of papers that cited this one
paper.externalIds 
text_example = paper.abstract
# paper.embedding
print(text_example)

In spite of many years of work by a multitude of organizations, a clear and simple standard for software engineering and management requirements and a method to assess their applicability to projects of various types and sizes remains elusive. From IEEE to CMMI to NASA's NPR 7150.2, there is no shortage of sources of information providing various types of requirements and standards for software engineering. Even a book on software project management for “dummies” approaches 400 pages. Wading through this information can dizzy the mind of even the most experienced software engineer; the newbie just trying to “do the right thing” will probably give up, open a text editor and start coding. This lack of clarity and simplicity perhaps goes a long way towards explaining why, in spite of this large body of work, there remains such an incredible variability in the knowledge and application of software engineering discipline not only from one organization to the next, but between groups within 

In [13]:
### Extract gz file information
text = []
with open("../mount-papers/fulltext/20230609_111759_00053_f77bs_0249e9d0-0537-4baa-bab3-78b0c18ce031") as f:
    for i in range(10,12):
        print(f.readline(), end='\n')
#     all_text = f.readlines()
#     all_text = json.loads(f.readlines())
#     print(all_text)
#     for lines in all_text:
#         listed_line = list(lines)
#         find_text = listed_line["text"]
#         print(listed_line)
    f.close()

{"corpusid":258426684,"externalids":{"arxiv":"2305.00178","mag":null,"acl":null,"pubmed":null,"pubmedcentral":null,"dblp":null,"doi":"10.1007/s13163-023-00468-z"},"content":{"source":{"pdfurls":["https://export.arxiv.org/pdf/2305.00178v1.pdf"],"pdfsha":"2f6878905f3611424e946ba149e3d5f6ab796771","oainfo":null},"text":"\nALGEBRAIC VALUATION RING EXTENSIONS AS LIMITS OF COMPLETE INTERSECTION ALGEBRAS\n29 Apr 2023\n\nDorin Popescu \nALGEBRAIC VALUATION RING EXTENSIONS AS LIMITS OF COMPLETE INTERSECTION ALGEBRAS\n29 Apr 2023immediate extensionspseudo convergent sequencespseudo limitssmooth morphismscomplete intersection algebras 2020 Mathematics Subject Classification: Primary 13F30Secondary 13A1813F2013B40\nWe show that an algebraic immediate valuation ring extension of characteristic p > 0 is a filtered union of complete intersection algebras of finite type.\n\nIntroduction\n\nB. Antieau and R. Datta have recently proven a positive characteristic analogue [1, Theorem 4.1.1] of Zariski's t

## Model Testing

In [18]:
max_sequence_length = 512
embedding_size = 200

In [19]:
T5tokens(text_example)

{'input_ids': [86, 17912, 13, 186, 203, 13, 161, 57, 3, 9, 16635, 13, 2371, 6, 3, 9, 964, 11, 650, 1068, 21, 889, 3867, 11, 758, 1502, 11, 3, 9, 1573, 12, 6570, 70, 1120, 2176, 2020, 12, 1195, 13, 796, 1308, 11, 4342, 3048, 3, 29748, 5, 1029, 23570, 12, 3, 5518, 7075, 12, 15971, 31, 7, 445, 5554, 489, 1808, 18189, 6, 132, 19, 150, 16316, 13, 2836, 13, 251, 1260, 796, 1308, 13, 1502, 11, 2443, 21, 889, 3867, 5, 1441, 3, 9, 484, 30, 889, 516, 758, 21, 105, 26, 30885, 153, 6315, 4837, 1688, 5, 3129, 26, 53, 190, 48, 251, 54, 1227, 19974, 8, 809, 13, 237, 8, 167, 1906, 889, 9739, 117, 8, 126, 4232, 131, 1119, 12, 105, 26, 32, 8, 269, 589, 153, 56, 1077, 428, 95, 6, 539, 3, 9, 1499, 6005, 11, 456, 3, 9886, 5, 100, 2136, 13, 12428, 11, 16538, 2361, 1550, 3, 9, 307, 194, 1587, 13834, 572, 6, 16, 17912, 13, 48, 508, 643, 13, 161, 6, 132, 3048, 224, 46, 3904, 27980, 16, 8, 1103, 11, 917, 13, 889, 3867, 7998, 59, 163, 45, 80, 1470, 12, 8, 416, 6, 68, 344, 1637, 441, 8, 337, 1470, 6, 42, 237, 344

In [20]:
bio_bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [21]:
### Instead of summarization can be used for classification of papers
def biobert_classifier(embedding_size = 200, input_dimensions = 3, hidden_layers = 0, max_sequence_length = 512, learning_rate=0.01):
  input_ids = tf.keras.layers.Input(shape=embedding_size, name="input_ids")
  token_type_ids = tf.keras.layers.Input(shape=embedding_size, name="token_type_id")
  attention_mask = tf.keras.layers.Input(shape=embedding_size, name="attention_mask")

  model_inputs = {"input_ids": input_ids,
                  "token_type_ids": token_type_ids,
                  "attention_mask": attention_mask}

  embedding_matrix = tf.keras.layers.Embedding(200)

  normalization_layer = tf.keras.layers.BatchNormalization()
    
  attention_layer = tf.keras.layers.Attention()

  pooler_layer = bio_bert_model(model_inputs)[0]

  dense_layer = tf.keras.layers.Dense(100, activation="relu")(pooler_layer)

  dropout_layer = tf.keras.layers.Dropout(0.3)(dense_layer)

  final_layer = tf.keras.layers.Dense(1, activation="relu")(dropout_layer)

  classification_layer = tf.keras.layers.Dense(1, activation='sigmoid')(final_layer)

  model = tf.keras.Model(inputs = [input_ids, token_type_ids, attention_mask], outputs = [classification_layer])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
                loss = tf.keras.losses.BinaryCrossentropy, 
                metrics = [tf.keras.metrics.Accuracy, tf.keras.metrics.Precision])
  
  return model

In [22]:
summary_task_prefix = "Summarize :"
qa_task_prefix = "Question :"

In [47]:
### T5 Abstractive Text Summarization Model
def t5summary_model(tokenizer, text, t5model):
    summarize = "summarize: "
    encoding = tokenizer([summarize+text], return_tensors='tf')
    output = t5model.generate(encoding.input_ids,
                             num_beams=3,
                            no_repeat_ngram_size=2,
                              top_k=10,
                              top_p=80,
                              max_length=50,
                              min_length=30
                             )
    print([tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in output])
#     print(summary)

In [41]:
T5Abstract_model

## Training New Models

In [45]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 11.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=282478fa0e8622b571e0e77f30ef2eb351f7ab7321eeb44d033344b4a8e0f9e9
  Stored in directory: /home/brathwaiten16_berkeley_edu/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [46]:
from transformers import BioGptModel, BioGptConfig, BioGptTokenizer
biogpttokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
biogptmodel = BioGptModel.from_pretrained("microsoft/biogpt")

Some weights of the model checkpoint at microsoft/biogpt were not used when initializing BioGptModel: ['output_projection.weight']
- This IS expected if you are initializing BioGptModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BioGptModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
t5summary_model(T5tokens, text_example, T5Abstract_model)

['a clear and simple standard for software engineering and management requirements remains elusive. the paper will make an admittedly bold and brash attempt to boil it all down into something anyone can understand, hopefully resulting in an brief reference']


### Q&A Model

In [ ]:
### BioBERT or BERT Q&A or Clincal-T5-Large

### Extractive Summary Model

In [ ]:
### T5 or T5v1 or Clincal-T5-Large or Bio-GPT

### Classification Model

In [ ]:
### BERT or Bio-GPT

### NER Model

In [ ]:
### BioELECTRA